In [2]:
import sys

sys.path.append('/Users/floyd/Desktop/DeepIV/deepiv')


from __future__ import print_function
import warnings

from models import Treatment, Response
import architectures as architectures
import densities as densities

from keras.layers import Input, Dense, Reshape
from keras.models import Model
from keras.layers.merge import Concatenate
import keras.backend as K

import numpy

import data_generator

Using TensorFlow backend.


In [3]:
def conv_embedding(images, output, 
                   other_features = [], 
                   dropout_rate=0.1,
                   embedding_dropout=0.1, 
                   embedding_l2=0.05, 
                   constrain_norm=True):
    
    print("Building conv net")
    x_embedding = architectures.convnet(images, 
                                        Dense(64, activation='linear'),
                                        dropout_rate=embedding_dropout,
                                        activations='relu',
                                        l2_rate=embedding_l2, 
                                        constrain_norm=constrain_norm)

    if len(other_features) > 0: # if other_features not empty
        embedd = Concatenate(axis=1)([x_embedding] + other_features)# other_features: list
    else:
        embedd = x_embedding
    out = architectures.feed_forward_net(embedd, 
                                         output,
                                         hidden_layers=[32],
                                         dropout_rate=dropout_rate,
                                         activations='relu', 
                                         constrain_norm=constrain_norm)
    return out

In [4]:
n = 5000
dropout_rate = min(1000./(1000. + n), 0.5)

embedding_dropout = 0.1
embedding_l2 = 0.1
epochs = int(1500000./float(n))
batch_size = 100

In [5]:

x, z, t, y, g_true = data_generator.demand(n=n, seed=1, ypcor=0.5, use_images=True, test=False)

print("Data shapes:\n\
Features:{x},\n\
Instruments:{z},\n\
Treament:{t},\n\
Response:{y}".format(**{'x':x.shape, 'z':z.shape,
                        't':t.shape, 'y':y.shape}))

11493376/11490434 [==============================] - 17s 2us/step
Data shapes:
Features:(5000, 785),
Instruments:(5000, 1),
Treament:(5000, 1),
Response:(5000, 1)


In [7]:
# Build and fit treatment model
if K.image_data_format() == "channels_first":
    image_shape = (1, 28, 28)
else:
    image_shape = (28, 28, 1)
    
    

images = Input(shape=(28 * 28,), name='treat_images')
image_reshaped = Reshape(image_shape)(images) # reshape
time = Input(shape=(1,), name='treat_time')
instruments = Input(shape=(z.shape[1],), name='treat_instruments')

mix_gaussian_output = lambda x: densities.mixture_of_gaussian_output(x, 10)

treatment_output = conv_embedding(image_reshaped, mix_gaussian_output,
                                  [time, instruments], 
                                  dropout_rate=dropout_rate,
                                  embedding_dropout=embedding_dropout,
                                  embedding_l2=embedding_l2)


treatment_model = Treatment(inputs=[instruments, time, images], outputs=treatment_output)
treatment_model.compile('adam',
                        loss="mixture_of_gaussians",
                        n_components=10)

treatment_model.fit([z, x[:,0:1], x[:,1:]], t, epochs=epochs, batch_size=batch_size)
treatment_model.save("demand_mnist_treatment.hd5")



Building conv net
Epoch 1/300
5000/5000 [==============================] - 12s 2ms/step - loss: 12.9387
Epoch 2/300
5000/5000 [==============================] - 11s 2ms/step - loss: 3.9195
Epoch 3/300
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3210
Epoch 4/300
5000/5000 [==============================] - 11s 2ms/step - loss: 1.8213
Epoch 5/300
5000/5000 [==============================] - 11s 2ms/step - loss: 1.4383
Epoch 6/300
5000/5000 [==============================] - 11s 2ms/step - loss: 1.2720
Epoch 7/300
5000/5000 [==============================] - 11s 2ms/step - loss: 1.1565
Epoch 8/300
5000/5000 [==============================] - 11s 2ms/step - loss: 1.0674
Epoch 9/300
5000/5000 [==============================] - 12s 2ms/step - loss: 1.0518
Epoch 10/300
5000/5000 [==============================] - 12s 2ms/step - loss: 1.0289
Epoch 11/300
5000/5000 [==============================] - 12s 2ms/step - loss: 1.0077
Epoch 12/300
5000/5000 [====================

5000/5000 [==============================] - 11s 2ms/step - loss: 0.6030
Epoch 97/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5851
Epoch 98/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.6009
Epoch 99/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5865
Epoch 100/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.6069
Epoch 101/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5807
Epoch 102/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5893
Epoch 103/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5927
Epoch 104/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5912
Epoch 105/300
5000/5000 [==============================] - 12s 2ms/step - loss: 0.5856
Epoch 106/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.5793
Epoch 107/300
5000/5000 [==============================] - 1

5000/5000 [==============================] - 11s 2ms/step - loss: 0.4652
Epoch 191/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4611
Epoch 192/300
5000/5000 [==============================] - 12s 2ms/step - loss: 0.4742
Epoch 193/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4759
Epoch 194/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4699
Epoch 195/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4670
Epoch 196/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4702
Epoch 197/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4741
Epoch 198/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4575
Epoch 199/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4739
Epoch 200/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4626
Epoch 201/300
5000/5000 [==============================] 

5000/5000 [==============================] - 11s 2ms/step - loss: 0.4429
Epoch 285/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4418
Epoch 286/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4310
Epoch 287/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4388
Epoch 288/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4358
Epoch 289/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4422
Epoch 290/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4425
Epoch 291/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4236
Epoch 292/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4410
Epoch 293/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4404
Epoch 294/300
5000/5000 [==============================] - 11s 2ms/step - loss: 0.4403
Epoch 295/300
5000/5000 [==============================] 

In [ ]:

# Build and fit response model

treatment = Input(shape=(t.shape[1],), name="treatment")

out_res = conv_embedding(image_reshaped, Dense(1, activation='linear'), [time, treatment],
                dropout_rate=dropout_rate, embedding_dropout=embedding_dropout, embedding_l2=embedding_l2)

response_model = Response(treatment=treatment_model,
                          inputs=[time, images, treatment],
                          outputs=out_res)

response_model.compile('adam', loss='mse', unbiased_gradient=True, batch_size=batch_size)
response_model.fit([z, x[:,0:1], x[:,1:]], y, epochs=epochs, verbose=1,
                   batch_size=batch_size, samples_per_batch=2)
treatment_model.save("demand_mnist_response.hd5")



In [ ]:
def datafunction(n, s, images=True, test=False):
    return data_generator.demand(n=n, seed=s, ypcor=0.5, use_images=images, test=test)

oos_perf = data_generator.monte_carlo_error(lambda x,z,t: response_model.predict([x[:,0:1], x[:,1:],t]), datafunction, has_latent=True, debug=False)
print("Out of sample performance evaluated against the true function: %f" % oos_perf)

